In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import scipy
import pdb
import matplotlib.pyplot as plt

In [8]:
from neurosim.models.ssr import StateSpaceRealization as SSR

### Goal 1: What is the Kalman Filter of an all pass system?...All pass systems carry no PI

In [5]:
size =10
A = np.random.normal(scale=1/(2 * np.sqrt(size)), size=(size, size))
while max(np.abs(np.linalg.eigvals(A))) > 0.99:
    A = np.random.normal(scale=1/(2 * np.sqrt(size)), size=(size, size))

C = scipy.stats.ortho_group.rvs(size)[:, 0:2].T

In [6]:
H = scipy.linalg.solve_discrete_lyapunov(A.T, -C.T @ C)
D = np.eye(C.shape[0]) + C @ np.linalg.inv(H) @ np.linalg.inv(np.eye(C.shape[1]) - A.T) @ C.T
B = np.linalg.inv(H) @ np.linalg.inv(A.T) @ C.T @ D

In [7]:
ssr = SSR(A=A, B=B, C=C, D=D)

In [8]:
z = np.exp(1j * np.linspace(0, 2*np.pi))

In [46]:
np.allclose(H, np.conjugate(H.T))

True

In [10]:
# Verify all pass
[ssr.transfer_fn(np.conjugate(zz)).T @ ssr.transfer_fn(zz) for zz in z]

[array([[ 9.99999998e-01+0.j, -2.43277057e-08+0.j],
        [-2.43277057e-08+0.j,  1.00000005e+00+0.j]]),
 array([[ 9.99999998e-01+0.00000000e+00j, -2.43277917e-08-2.64094302e-13j],
        [-2.43277917e-08+2.64094302e-13j,  1.00000005e+00+0.00000000e+00j]]),
 array([[ 9.99999998e-01+0.000000e+00j, -2.43277435e-08-3.378825e-13j],
        [-2.43277435e-08+3.378825e-13j,  1.00000005e+00+0.000000e+00j]]),
 array([[ 9.99999998e-01+0.0000000e+00j, -2.43276002e-08-3.0644931e-13j],
        [-2.43276002e-08+3.0644931e-13j,  1.00000005e+00+0.0000000e+00j]]),
 array([[ 9.99999998e-01+0.00000000e+00j, -2.43274615e-08-1.47312718e-13j],
        [-2.43274615e-08+1.47312718e-13j,  1.00000005e+00+0.00000000e+00j]]),
 array([[ 9.99999998e-01+0.00000000e+00j, -2.43272250e-08+7.30804306e-14j],
        [-2.43272250e-08-7.30804306e-14j,  1.00000005e+00+0.00000000e+00j]]),
 array([[ 9.99999998e-01+0.00000000e+00j, -2.43270194e-08+2.10831352e-13j],
        [-2.43270194e-08-2.10831352e-13j,  1.00000005e+00+0.

In [33]:
# What is the Kalman Filter?
ssr.solve_min_phase()
ssr.solve_max_phase()

In [12]:
ssr.Bmin

array([[-0.24127332, -0.06252671],
       [ 0.25307728,  0.05118513],
       [ 0.4085069 ,  0.05519663],
       [-0.30063026, -0.01534044],
       [ 0.27348306,  0.03744378],
       [ 0.54967689,  0.086059  ],
       [ 0.10316883,  0.01435869],
       [ 0.07299752,  0.00602682],
       [ 0.11373645, -0.00330699],
       [ 0.38932857,  0.06500172]])

In [13]:
ssr.B

array([[-104.07864782,  -36.66853396],
       [  71.5585368 ,   80.86870551],
       [ -41.1827777 ,  295.16025656],
       [  60.94261274, -256.7707948 ],
       [   2.19813046,  168.06557222],
       [  42.58560726,  296.18129193],
       [ -29.21584791,   92.95246756],
       [ -28.1823672 ,   74.701012  ],
       [ -35.30267861,  112.59055941],
       [  27.01165889,  211.38309994]])

In [14]:
ssr.Dmin

array([[ 0.99796489, -0.00367639],
       [-0.00367639,  0.99329725]])

In [15]:
ssr.D

array([[-0.00176139, -0.00159601],
       [-0.0005397 , -0.00020096]])

In [16]:
ssr.Bmin @ np.linalg.inv(ssr.Dmin)

array([[-0.24200053, -0.06384433],
       [ 0.25378666,  0.05246984],
       [ 0.40955025,  0.05708492],
       [-0.30130432, -0.01655914],
       [ 0.27418337,  0.03871126],
       [ 0.55112451,  0.08867955],
       [ 0.10343388,  0.01483842],
       [ 0.07316973,  0.0063383 ],
       [ 0.11395768, -0.00290752],
       [ 0.39036891,  0.06688518]])

In [18]:
np.linalg.eigvals(ssr.P)

array([3.51695055e+05, 2.72047614e+04, 6.06659318e+03, 3.95012636e+02,
       9.80567170e+01, 2.48580239e+01, 6.75939850e+00, 5.20659671e+00,
       8.81082077e-01, 9.19027935e-01])

In [30]:
_, S, _ = np.linalg.svd(ssr.transfer_fn(z[8]))
S

array([1.00000003, 0.99999999])

In [32]:
np.linalg.eigvals(ssr.Pmin)

array([1.01162686e+00, 1.14122026e-01, 6.78702494e-03, 2.16832706e-03,
       1.82411617e-04, 8.56808605e-05, 3.76663922e-05, 1.87663249e-05,
       8.22860923e-06, 1.64441402e-06])

In [34]:
np.linalg.eigvals(ssr.Pmax)

array([1.13496730e+00, 1.08810012e+00, 1.92063118e-01, 1.47941988e-01,
       4.02283428e-02, 1.01981579e-02, 2.53155737e-03, 1.64836903e-04,
       3.67581012e-05, 2.84336753e-06])

In [35]:
np.linalg.eigvals(ssr.Pmin @ ssr.Pmax)

array([2.59160010e-02, 4.60280846e-05, 8.03344335e-06, 9.79967167e-07,
       5.72941148e-06, 5.93799639e-06, 5.93793850e-06, 4.45999052e-08,
       4.33574559e-08, 4.34693875e-08])

In [38]:
ssr.pi(11)

1.401217881830807e-05

In [39]:
Q = scipy.linalg.solve_discrete_lyapunov(ssr.A, ssr.C.T @ ssr.C)

In [41]:
np.linalg.eigvals(ssr.P @ Q)

array([2.46536501e+04, 4.22950065e+03, 1.50116447e+02, 6.13001217e+00,
       1.01738346e+00, 9.96439076e-01, 2.85131837e-01, 1.76346241e-03,
       8.78914675e-04, 6.97734669e-06])

In [43]:
[ssr.transfer_fn(zz) @ ssr.transfer_fn(np.conjugate(zz)).T for zz in z]

[array([[ 9.99999998e-01+0.j, -2.43277062e-08+0.j],
        [-2.43277062e-08+0.j,  1.00000005e+00+0.j]]),
 array([[ 1.00000000e+00+0.00000000e+00j, -2.22359290e-08-1.47668918e-08j],
        [-2.22359290e-08+1.47668918e-08j,  1.00000004e+00+0.00000000e+00j]]),
 array([[ 1.00000001e+00+0.00000000e+00j, -1.67999871e-08-2.56527345e-08j],
        [-1.67999871e-08+2.56527345e-08j,  1.00000004e+00+0.00000000e+00j]]),
 array([[ 1.00000001e+00+0.00000000e+00j, -9.66853330e-09-3.15764733e-08j],
        [-9.66853330e-09+3.15764733e-08j,  1.00000003e+00+0.00000000e+00j]]),
 array([[ 1.00000002e+00+0.00000000e+00j, -2.11590963e-09-3.36472423e-08j],
        [-2.11590963e-09+3.36472423e-08j,  1.00000002e+00+0.00000000e+00j]]),
 array([[1.00000002e+00+0.00000000e+00j, 5.11414203e-09-3.33635473e-08j],
        [5.11414203e-09+3.33635473e-08j, 1.00000002e+00+0.00000000e+00j]]),
 array([[1.00000002e+00+0.00000000e+00j, 1.14112633e-08-3.17685112e-08j],
        [1.14112633e-08+3.17685112e-08j, 1.00000002e+0

In [49]:
np.linalg.eigvals(A - B @ np.linalg.inv(D) @ C)

array([19.41519681+0.j        , -6.54578875+0.j        ,
        0.51866263+5.85169824j,  0.51866263-5.85169824j,
       -2.33241443+3.0615396j , -2.33241443-3.0615396j ,
        3.89614756+0.j        ,  0.5560332 +2.04128407j,
        0.5560332 -2.04128407j,  1.73941871+0.j        ])

In [50]:
np.linalg.eigvals(np.linalg.inv(A).T)

array([19.41519962+0.j        , -6.54578875+0.j        ,
        0.51866203+5.85169923j,  0.51866203-5.85169923j,
       -2.33241481+3.0615393j , -2.33241481-3.0615393j ,
        0.55603314+2.04128381j,  0.55603314-2.04128381j,
        3.8961469 +0.j        ,  1.73941863+0.j        ])

In [53]:
Q = scipy.linalg.solve_discrete_lyapunov(A.T, C.T @ C)

In [55]:
ssr.P @ Q

array([[ 1.00000005e+00,  6.74485818e-08,  1.00687885e-08,
         4.65076937e-08,  2.18364562e-08, -6.74890543e-09,
        -2.75315415e-08, -4.91365881e-09,  5.89038223e-08,
         3.50223672e-09],
       [ 4.31339231e-09,  1.00000001e+00,  5.68570613e-09,
         1.09866960e-08, -5.66228664e-09, -1.44850674e-08,
         1.73808985e-08,  8.32744718e-09,  4.38760992e-08,
         1.75759851e-09],
       [-4.21273398e-08, -8.55743565e-09,  1.00000009e+00,
        -1.33522917e-08, -1.89156708e-08, -7.08532752e-08,
        -7.40155883e-08,  4.30131877e-08,  1.18268360e-07,
        -6.38192432e-09],
       [ 6.84622137e-09,  1.72331056e-08, -1.83886186e-08,
         9.99999999e-01,  5.60476110e-09,  1.94950189e-08,
        -5.28598321e-09, -4.77439244e-09, -1.15436933e-07,
        -9.48375600e-09],
       [-4.99483122e-08, -6.25050234e-08,  3.05144567e-08,
        -4.31459739e-08,  9.99999975e-01, -1.84591045e-08,
        -3.21485913e-08,  1.40694283e-08,  1.87143598e-08,
         9.

In [56]:
# All pass systems have positive group delay
ssr.spectral_density(z)

array([[[ 9.99999998e-01+0.00000000e+00j,
         -2.43277062e-08+0.00000000e+00j],
        [-2.43277062e-08+0.00000000e+00j,
          1.00000005e+00+0.00000000e+00j]],

       [[ 1.00000000e+00+0.00000000e+00j,
         -2.22359290e-08-1.47668918e-08j],
        [-2.22359290e-08+1.47668918e-08j,
          1.00000004e+00+0.00000000e+00j]],

       [[ 1.00000001e+00+0.00000000e+00j,
         -1.67999871e-08-2.56527345e-08j],
        [-1.67999871e-08+2.56527345e-08j,
          1.00000004e+00+0.00000000e+00j]],

       [[ 1.00000001e+00-3.77302356e-15j,
         -9.66854268e-09-3.15764727e-08j],
        [-9.66860530e-09+3.15764257e-08j,
          1.00000003e+00+2.67147415e-15j]],

       [[ 1.00000002e+00+0.00000000e+00j,
         -2.11590963e-09-3.36472423e-08j],
        [-2.11590963e-09+3.36472423e-08j,
          1.00000002e+00+0.00000000e+00j]],

       [[ 1.00000002e+00+0.00000000e+00j,
          5.11414203e-09-3.33635473e-08j],
        [ 5.11414203e-09+3.33635473e-08j,
          1.0

In [58]:
x = ssr.trajectory(int(1e5))

In [61]:
from dca.cov_util import calc_pi_from_data

In [65]:
calc_pi_from_data(x, 2)

1.4395752900654745e-05

In [ ]:
# All-pass functions have no predictive information?

### Goal 2: How do various loop architectures induce a Schur form?

In [1]:
# How do various loop architectures induce a Schur form?

In [13]:
A1 = np.array([[0.8, 0.25], [-0.25, 0.75]])

In [14]:
scipy.linalg.schur(A1)

(array([[ 0.775,  0.275],
        [-0.225,  0.775]]),
 array([[ 0.70710678,  0.70710678],
        [-0.70710678,  0.70710678]]))

In [15]:
# Need to answer a series of empirical questions 
# (1) When we restrict to the image of C, what do things look like in the schur basis of the different methods? 
# (2) How do the minimum phase versions of B load onto the Schur vectors?
# (3) What are the dynamics and robustness of the marginal dynamics? What is the role played by the rest of the space?
# (4) Does group delay provide a means of measuring the extent of feedback?

In [16]:
alpha = 0.1
beta = 0.1

In [17]:
A = np.array([[0.85, alpha, 0], [0, 0.8, alpha], [beta, 0, 0.75]])

In [18]:
scipy.linalg.schur(A)

(array([[ 0.90831564,  0.02061113, -0.03698739],
        [ 0.        ,  0.74584218, -0.05654185],
        [ 0.        ,  0.11140789,  0.74584218]]),
 array([[ 0.75830454,  0.60665228,  0.23863621],
        [ 0.44221012, -0.20972108, -0.87204775],
        [ 0.47898271, -0.76680511,  0.42730023]]))